In [212]:
import numpy as np
with open("Sistemas de prueba\IEEE34.atp", "r+") as f:
    lines = f.readlines()
lines_copy = lines.copy()

# Extraer Lineas de las cargas en archivo
three_phase_loads = []
single_phase_loads = []
switches = []
for idx, line in enumerate(lines_copy):
    
    if "C Load" in line:
        if "3f" in line:
            three_phase_loads.append(lines_copy[idx:idx+4])
            
        else:
            single_phase_loads.append(lines_copy[idx:idx+2])
ti = 0.01
tf = 0.06


In [213]:
three_phase_ti = np.random.uniform(ti, tf, (len(three_phase_loads),3))
three_phase_ti = np.around(three_phase_ti, 5)
single_phase_ti = np.random.uniform(ti, tf, len(single_phase_loads))
single_phase_ti = np.around(single_phase_ti, 5)

In [214]:
# Iterar por cada Carga
from utils.components import single_phase_switch, three_phase_switch
for load_idx, loads in enumerate(three_phase_loads):
    print("\n")
    switch = three_phase_switch.copy()
    switch_node = []
    for idx, load in enumerate(loads):
        print(load, end='')
        
        if idx == 0:
            load_node = f"L{load_idx}"
            loads[idx] = f"{load.strip()}-Load Change".ljust(80)+'\n'
            switch_node.append(f"{load.strip()} - switch".ljust(80)+'\n')
            continue

        node_1 = load[2:8]
        phase_1 = node_1.strip()[-1]
        phase_2 = ''
        node_2 = load[8:14].strip()
        R = load[30:42]
        X = load[46:58]
        load_node_phase = f"{load_node}{phase_1}".center(6)
        load_node_phase_2 = ''.center(6)
        if node_2:
            phase_2 = node_2[-1]
            load_node_phase_2 = f"{load_node}{phase_2}".center(6)
        node_2 = node_2.center(6)
        switch_ln = switch[idx]
        ti = str(three_phase_ti[load_idx, idx-1]).center(10)
        tff = str(tf).center(10)
        
        
        # Create new Lines
        switch[idx] = f"{switch_ln[:2]}{node_1}{load_node_phase}{ti}{tff}{switch_ln[34:]}"
        loads[idx] = f"{load[:2]}{load_node_phase}{load_node_phase_2}{load[14:]}"
        switch_node.append(switch[idx])
    switches.append(switch_node)
        

        





C Load 860 3fy
  X0072A                      6300.9146341    5040.7317073                     0
  X0072B                      6300.9146341    5040.7317073                     0
  X0072C                      6300.9146341    5040.7317073                     0


C Load 840 3fy
  N840A                       14307.923077    11128.384615                     0
  N840B                       14307.923077    11128.384615                     0
  N840C                       14307.923077    11128.384615                     0


C Load 844 3f
  N844A                       953.86153846    741.89230769                     0
  N844B                       953.86153846    741.89230769                     0
  N844C                       953.86153846    741.89230769                     0


C Load 830 3f
  N830A N830B                      49600.8         24800.4                     0
  N830B N830C                      49600.8         24800.4                     0
  N830C N830A                 21379.655172 

In [215]:
for loads in three_phase_loads:
    print('\n')
    for load in loads:
        print(load, end='')



C Load 860 3fy-Load Change                                                      
   L0A                        6300.9146341    5040.7317073                     0
   L0B                        6300.9146341    5040.7317073                     0
   L0C                        6300.9146341    5040.7317073                     0


C Load 840 3fy-Load Change                                                      
   L1A                        14307.923077    11128.384615                     0
   L1B                        14307.923077    11128.384615                     0
   L1C                        14307.923077    11128.384615                     0


C Load 844 3f-Load Change                                                       
   L2A                        953.86153846    741.89230769                     0
   L2B                        953.86153846    741.89230769                     0
   L2C                        953.86153846    741.89230769                     0


C Load 830 3f-Load C

In [218]:
print(len(switches[0]))
print(len(three_phase_loads[0]))

4
4


In [210]:
for switch in switches:
    print('\n')
    for line in switch:
        print(line, end='')



C Load 860 3fy - switch                                                         
  X0072A L0A   0.02343     0.06                                                0
  X0072B L0B   0.02259     0.06                                                0
  X0072C L0C   0.05593     0.06                                                0


C Load 840 3fy - switch                                                         
  N840A  L1A   0.02462     0.06                                                0
  N840B  L1B   0.02758     0.06                                                0
  N840C  L1C   0.04223     0.06                                                0


C Load 844 3f - switch                                                          
  N844A  L2A   0.01267     0.06                                                0
  N844B  L2B   0.05643     0.06                                                0
  N844C  L2C   0.01538     0.06                                                0


C Load 830 3f - swit